In [60]:
import pandas as pd
import numpy as np
from SERGIO.SERGIO.sergio import sergio
from GENIE3.GENIE3 import *
import MAGIC.magic as magic
import SAUCIE.SAUCIE as SAUCIE
import scScope.scscope.scscope as DeepImpute
import deepimpute.deepimpute as deepimpute

import os
import h5py
import importlib
import scprep
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm

from scipy.sparse import csr_matrix, csc_matrix

In [61]:
#file_path = './network_data/HU_0107_Bladder_GSE134355_gene_count.h5'
file_path = './network_data/HU_0379_Blood_GSE172495_gene_count.h5'
#file_path = './network_data/HU_0142_Breast_GSE138536_gene_count.h5'

with h5py.File(file_path, 'r') as h5file:
    if 'matrix' in h5file:
        shape = tuple(h5file['matrix']['shape'][:])
        print("Shape of matrix: ", shape)

        data = h5file['matrix']['data'][:]
        indices = h5file['matrix']['indices'][:]
        indptr = h5file['matrix']['indptr'][:]

        matrix = csc_matrix((data, indices, indptr), shape=shape)
        dense_matrix = matrix.toarray()
        df = pd.DataFrame(data=dense_matrix)
        df.replace(0, 1e-5, inplace=True)

Shape of matrix:  (11738, 1253)


In [62]:
df

,0,1,2,3,4,5,6,7,8,9,...,1243,1244,1245,1246,1247,1248,1249,1250,1251,1252
0,0.000010,0.000010,0.00001,0.00001,0.00001,0.000010,0.00001,0.00001,0.00001,0.00001,...,0.00001,0.00001,0.00001,0.000010,0.000010,0.00001,0.00001,0.00001,0.000010,0.00001
1,0.000010,0.000010,0.00001,0.00001,0.00001,0.000010,0.00001,0.00001,0.00001,0.00001,...,0.00001,0.00001,0.00001,0.000010,2.412415,0.00001,0.00001,0.00001,0.000010,0.00001
2,0.000010,0.000010,0.00001,0.00001,0.00001,0.000010,0.00001,0.00001,0.00001,0.00001,...,0.00001,0.00001,0.00001,0.000010,0.000010,0.00001,0.00001,0.00001,0.000010,0.00001
3,1.525828,1.476872,0.00001,0.00001,0.00001,0.000010,0.00001,0.00001,0.00001,0.00001,...,0.00001,0.00001,0.00001,0.000010,0.000010,0.00001,0.00001,0.00001,0.000010,0.00001
4,0.000010,0.000010,0.00001,0.00001,0.00001,0.000010,0.00001,0.00001,0.00001,0.00001,...,0.00001,0.00001,0.00001,0.000010,0.000010,0.00001,0.00001,0.00001,0.000010,0.00001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11733,0.000010,0.000010,0.00001,0.00001,0.00001,0.000010,0.00001,0.00001,0.00001,0.00001,...,0.00001,0.00001,0.00001,0.000010,0.000010,0.00001,0.00001,0.00001,0.000010,0.00001
11734,0.000010,0.000010,0.00001,0.00001,0.00001,2.056643,0.00001,0.00001,1.89862,0.00001,...,0.00001,0.00001,0.00001,2.537908,0.000010,0.00001,0.00001,0.00001,0.000010,0.00001
11735,0.000010,0.000010,0.00001,0.00001,0.00001,0.000010,0.00001,0.00001,1.89862,0.00001,...,0.00001,0.00001,0.00001,0.000010,0.000010,0.00001,0.00001,0.00001,0.000010,0.00001
11736,0.000010,0.000010,0.00001,0.00001,0.00001,0.000010,0.00001,0.00001,0.00001,0.00001,...,0.00001,0.00001,0.00001,0.000010,0.000010,0.00001,0.00001,0.00001,2.809927,0.00001


In [63]:
def run_saucie(x_path, y_path, ind, save_path):
    #reload_modules('tensorflow.compat')
    tf = importlib.import_module('tensorflow.compat.v1')
    #importlib.reload(SAUCIE)
    tf.disable_v2_behavior()
    print("loading data")
    y = np.transpose(np.load(y_path))
    x = np.transpose(np.load(x_path))
    print("reset graph")
    tf.reset_default_graph()
    print("Initialize saucie")
    saucie = SAUCIE.SAUCIE(y.shape[1])
    print("Load saucie")
    loadtrain = SAUCIE.Loader(y, shuffle=True)
    print("Train saucie")
    saucie.train(loadtrain, steps=1000)

    loadeval = SAUCIE.Loader(y, shuffle=False)
    # embedding = saucie.get_embedding(loadeval)
    # number_of_clusters, clusters = saucie.get_clusters(loadeval)
    rec_y = saucie.get_reconstruction(loadeval)
    save_str = '/yhat_SAUCIE'
    np.save(save_path + save_str, rec_y)

def run_deepImpute(x_path, y_path, ind, save_path):
    #reload_modules('tensorflow.compat')
    importlib.invalidate_caches()
    multinet = importlib.import_module('deepimpute.deepimpute.multinet')
    importlib.reload(multinet)
    tf = importlib.import_module('tensorflow.compat.v1')
    #tf = importlib.import_module('tensorflow')
    tf.init_scope()
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    y = np.transpose(np.load(y_path))
    y = 10 ** y
    y = pd.DataFrame(y)
    x = np.transpose(np.load(x_path))
    x = pd.DataFrame(x)
    multinet = multinet.MultiNet()
    multinet.fit(y,cell_subset=1,minVMR=0.5)
    imputedData = multinet.predict(y)
    yhat_deepimpute = imputedData.to_numpy()
    save_str = '/yhat_deepImpute'
    np.save(save_path + save_str, yhat_deepimpute)

def run_magic(x_path, y_path, ind, save_path):
    print(x_path, y_path)
    y = np.transpose(np.load(y_path))
    x = np.transpose(np.load(x_path))
    
    y_hat = scprep.filter.filter_rare_genes(y, min_cells=5)
    y_norm = scprep.normalize.library_size_normalize(y_hat)
    y_norm = scprep.transform.sqrt(y_norm)

    for t_val in [7]:
        magic_op = magic.MAGIC(
            t=t_val,
            n_pca=20,
            n_jobs=-1,
        )
        y_hat = magic_op.fit_transform(y_norm, genes='all_genes')
        save_str = '/yhat_MAGIC_t_' + str(t_val)
        np.save(save_path + save_str, y_hat)

def run_scScope(x_path, y_path, ind, save_path):
    y = np.transpose(np.load(y_path))
    x = np.transpose(np.load(x_path))
    DI_model = DeepImpute.train(
          y,
          15,
          use_mask=True,
          batch_size=64,
          max_epoch=1000,
          epoch_per_check=100,
          T=2,
          exp_batch_idx_input=[],
          encoder_layers=[],
          decoder_layers=[],
          learning_rate=0.0001,
          beta1=0.05,
          num_gpus=1)
    latent_code, rec_y, _ = DeepImpute.predict(y, DI_model, batch_effect=[])
    save_str = '/yhat_scScope'
    np.save(save_path + save_str, rec_y)

In [64]:
def create_graph(data, cutoff=0.95, method_name=None):
    print(method_name, cutoff, data.shape)
    edges = pd.DataFrame(data)
    values = edges.values.flatten()
    threshold = np.quantile(values, cutoff)

    G = nx.DiGraph()
    mask = edges >= threshold
    rows, cols = np.where(mask)
    edge_count = 0
    for row, col in tqdm(zip(rows, cols), total=len(rows)):
        weight = edges.iat[row, col]
        gene_1 = edges.index[row]
        gene_2 = edges.columns[col]
        G.add_edge(gene_1, gene_2, weight=weight)
        edge_count += 1
    print(edge_count)
    
    pos = nx.spring_layout(G)
    es, weights = zip(*nx.get_edge_attributes(G, 'weight').items())
    nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=50)
    nx.draw_networkx_edges(G, pos, edgelist=es, edge_color=weights, width=1, edge_cmap=plt.cm.Blues)

    if method_name is None:
        plt.title('Top GRN Inference Edges')
    else:
        plt.title('Top GRN Inference Edges with Imputation Method ' + method_name)
    plt.axis('off')
    plt.show()

In [69]:
def run_GRN_and_graph(data, n_genes=100):
    print("Num genes:", n_genes)
    sampled = data.sample(n=n_genes, random_state=42)
    np.save('./zero_imputation_experiments/sampled.npy', sampled)
    for i, method in enumerate([None, 'SAUCIE', 'scScope', 'DeepImpute', 'MAGIC']):
        print(i, method)
        if method is None:
            x = np.transpose(sampled.values)
        elif method == 'SAUCIE':
            print("Running SAUCIE")
            run_saucie('./zero_imputation_experiments/sampled.npy', './zero_imputation_experiments/sampled.npy', i, './zero_imputation_experiments/')
            x = np.load('./zero_imputation_experiments/yhat_SAUCIE.npy')
        elif method == 'scScope':
            print("Running scScope")
            run_scScope('./zero_imputation_experiments/sampled.npy', './zero_imputation_experiments/sampled.npy', i, './zero_imputation_experiments/')
            x = np.load('./zero_imputation_experiments/yhat_scScope.npy')
        elif method == 'DeepImpute':
            print("Running DeepImpute")
            run_deepImpute('./zero_imputation_experiments/sampled.npy', './zero_imputation_experiments/sampled.npy', i, './zero_imputation_experiments/')
            x = np.load('./zero_imputation_experiments/yhat_deepImpute.npy')
        elif method == 'MAGIC':
            print("Running MAGIC")
            run_magic('./zero_imputation_experiments/sampled.npy', './zero_imputation_experiments/sampled.npy', i, './zero_imputation_experiments/')
            x = np.load('./zero_imputation_experiments/yhat_MAGIC_t_7.npy')
        x[x == 0] = 1e-5
        vim = GENIE3(x, nthreads=12, ntrees=100, regulators='all')
        create_graph(vim, cutoff=0.95, method_name=method)